
Ссылка на модель и веса: https://drive.google.com/drive/folders/1ex_av6ClcWs-SmaPwM_ROj2aiRWxu6Dj?usp=sharing

Ссылка на базу: https://drive.google.com/file/d/1MQ36P2OpQDoO62_nqkk7uuuKlPIcdCd5/view?usp=sharing



In [ ]:
#@title Подключить диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Запуск распознавания. Распознавание 20-минутного видео ~16 минут
from tensorflow.keras.models import load_model
import os,glob,cv2
from IPython.display import clear_output
from tqdm import tqdm
from natsort import natsorted
import numpy as np
from moviepy.editor import * 

modelpath = str(input('Введите путь к сохраненной модели без кавычек'))
model = load_model(modelpath)
clear_output()

files = glob.glob('/content/temp/*')
for f in files:
  os.remove(f)
files = glob.glob('/content/cut/*')
for f in files:
  os.remove(f)

def take_frames_from_video_by_seconds(model, video_path, divide_sec_on):
  second = 0 
  cap = cv2.VideoCapture(video_path)
  frameRate = cap.get(5)/divide_sec_on
  image_folder = 'temp'
  while cap.isOpened():
    frameId = cap.get(1)
    ret, frame = cap.read()
    if not ret:
      break
    if frameId%round(frameRate,0)==1:  
      filename =f"{round(second, 1)}.jpg";second+=(1/round(frameRate,0))
      cv2.imwrite(image_folder+'/'+ filename, cv2.resize(frame, (229,229)))
  cap.release()
  cv2.destroyAllWindows()
  return round(frameRate,0)
  
def predict_from_frames(frameRate, model=model, path='temp'):
  counter = 0
  predict = []
  temp = []
  for filename in tqdm(natsorted(os.listdir(path))):

      img = cv2.imread(path+'/'+filename)
      img = img/255.
      pred = model.predict(img[None, :,:,:])
      temp.append(np.argmax(pred[0]))
      counter+=1
      if counter>=frameRate:
        predict.append(temp)
        temp = []
        counter = 0
  return np.array(predict)

def get_timings_from_predict(predict, frameRate):
  seconds = []
  for s, pred in enumerate(predict):
    if sum(pred)<frameRate*0.6:
      seconds.append(s)
  return seconds

def preprocess_timings(list_of_seconds, bias): # bias - разница между планами. Не разрезать кусок видео, если между от джима до джима промежуток bias секунд(0 - полная нетерпимость к отсутствию джима)
  time_list = []
  temp = []
  for i in range(len(list_of_seconds)-1):
    temp.append(list_of_seconds[i])
    if list_of_seconds[i+1]-list_of_seconds[i]>bias+1:
      time_list.append([temp[0], temp[-1]+1])
      temp = []
  return time_list

def extract_samples(filepath, time_list):
  for timings in tqdm(time_list):
    clip = VideoFileClip(filepath) 
    clip = clip.subclip(timings[0], timings[1]) 
    clip.write_videofile(filename = f"cut/{timings[0]}.mp4") 
    clip.close()

def make_one_clip_from_cuts(cutdir="cut/"):
  clips =[]
  for root, dirs, files in os.walk(cutdir):
      files = natsorted(files)
      for file in tqdm(files):
        filePath = os.path.join(root, file)
        video = VideoFileClip(filePath)
        clips.append(video)
  final_clip = concatenate_videoclips(clips)
  final_clip.write_videofile('output.mp4')
  files = glob.glob('/content/temp/*')
  for f in files:
    os.remove(f)
  files = glob.glob('/content/cut/*')
  for f in files:
    os.remove(f)

def main(model=model, fps_division = 5, bias = 1):
  filepath = str(input("Введите путь к видео без кавычек"))
  try: os.mkdir('temp'); os.mkdir('cut')
  except Exception:
    pass
  clear_output()
  print('Подготовка к распознаванию')
  divided_fps = take_frames_from_video_by_seconds(model, filepath, fps_division)
  clear_output()
  print('Распознавание')
  predict = predict_from_frames(divided_fps)
  clear_output()
  print('Расчет длительности клипа')
  seconds = get_timings_from_predict(predict, divided_fps)
  time_list = preprocess_timings(seconds, bias)
  clear_output()
  print('Нарезка клипов')
  extract_samples(filepath, time_list)
  clear_output()
  print('Склеивание клипов.')
  make_one_clip_from_cuts()
  print('Расчетное время Джима в кадре:',round(len(seconds)/60,0), "минут", len(seconds)%60, 'секунд')

main()

  0%|          | 0/95 [00:00<?, ?it/s]

Склеивание клипов.


100%|██████████| 95/95 [00:24<00:00,  3.82it/s]

[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing audio in outputTEMP_MPY_wvf_snd.mp3



100%|██████████| 9924/9924 [00:20<00:00, 478.39it/s]

[MoviePy] Done.
[MoviePy] Writing video output.mp4



100%|██████████| 10791/10791 [03:13<00:00, 55.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

Расчетное время Джима в кадре: 7.0 минут 1 секунд
